# El corrido de Gregorio Cortez

## Mapping the manhunt of Rio Grande border folk hero Gregorio Cortez:  the largest manhunt in U.S. history.

### June 14, 1901 to June 22, 1901

### based on lyrics from the Corrido de Gregorio Cortez (espanol)

#### project author Diane Lopez

In [1]:
import voila 

In [2]:
import geopandas as gpd

import geograpy
from geograpy import extraction
from geograpy import places
import re
from geograpy.labels import Labels

import nltk
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
from nltk.corpus import ieer
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

import geopy

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\azg526\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from geopy.geocoders import Nominatim
import time


In [4]:
import pandas as pd

In [5]:
from shapely.geometry import Point
import shapely

In [6]:
import contextily as ctx

In [7]:
import matplotlib.pyplot as plt #to make sure there are no errors when plotting a graph

In [8]:
import plotly.express as px

In [9]:
import plotly.graph_objects as go
import numpy as np


In [10]:
from ipywidgets import interactive

In [14]:
import spacy
from spacy import displacy

ModuleNotFoundError: No module named 'thinc.neural'

Geoparsing: Finding places from the corrido lyrics

In [ ]:
#reading text file
text = open('../corridos\el_corrido_greg_cortez.txt', encoding="utf-8").read()
# Replace line breaks with spaces
text = text.replace('\n', ' ')

for number, sentence in enumerate(nltk.sent_tokenize(text)):
    print(number, sentence)
 # Break text into sentences
sentences = nltk.sent_tokenize(text)

In [ ]:
spacy.displacy.render(text, style ='ent')

In [ ]:
LOC=extraction.Extractor(text=text)
LOC.find_geoEntities()

print(LOC.places)

In [ ]:
pc = places.PlaceContext(LOC.places)

pc.set_countries()
print (pc.countries) #['United States']

pc.set_regions()
print(pc.regions) #['Texas'])

pc.set_cities()
print(pc.cities) #['Brownsville'])

print(pc.address_strings) #['Brownsville','Texas, United States'])

Laredo, Tejas

In [ ]:
geolocator = Nominatim(user_agent="CorridosMap")

In [ ]:
laredo=pc.cities[0]
print (laredo)

need to know the country code https://en.wikipedia.org/wiki/ISO_3166-1_alpha-2#Current_codes

In [ ]:
laredo_info = geolocator.geocode(laredo)
laredo_lat = (laredo_info.latitude) 
laredo_long= (laredo_info.longitude)
print(laredo_info)
print(laredo_long,',', laredo_lat)

Belmont, Tejas/Texas

In [ ]:
belmont='Belmont, Gonzales County,Texas'
#belmont=pc.cities[1]
belmont_info=geolocator.geocode(belmont)
belmont_lat = (belmont_info.latitude)
belmont_long= (belmont_info.longitude)
print(belmont_info)
print(belmont_long,',', belmont_lat)

Gonzales, Tejas

In [ ]:

#gonzales='Gonzales,Texas'
gonzales=pc.cities[2]
gonzales_info=geolocator.geocode(gonzales)
gonzales_lat = (gonzales_info.latitude) 
gonzales_long =(gonzales_info.longitude)
#gonzales=gonzales_info, gonzales_coordinates
print(gonzales_lat, ',', gonzales_long)
print(gonzales_info)

dataframe

In [ ]:
greg = pd.DataFrame({'Place':[gonzales, 'Belmont', laredo], 'State': ['Texas', 'Texas', 'Texas'], 
'Latitude':[gonzales_lat, belmont_lat, laredo_lat], 'Longitude': [gonzales_long, belmont_long, laredo_long]})

In [ ]:
greg.head()

Setting Geometry- https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_geometry.html#

Creating a GeoDataFrame from a DataFrame with coordinates https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html

In [ ]:
greg_gpd = gpd.GeoDataFrame(
    greg, geometry=gpd.points_from_xy(greg.Longitude, greg.Latitude),  crs="EPSG:4326")

figure out later shapely 2.0 

In [ ]:
print(greg_gpd.head())
#greg_gpd.plot()

In [ ]:
#greg_gpd_basemap, greg_gpd_extent = ctx.bounds2img(*greg_gpd.total_bounds, zoom=10,   
                                            #source=ctx.providers.OpenStreetMap.Mapnik )

In [ ]:
px.set_mapbox_access_token(open("mapboxtoken").read())

fig = px.scatter_mapbox(greg_gpd, 
                        color='Place',
                        animation_group='Place',
                        animation_frame=['Gonzales', 'Belmont', 'Laredo'],
                        lat=greg_gpd.geometry.y, 
                        lon=greg_gpd.geometry.x, 
                        hover_name='Place',
                        size='Latitude',
                        zoom=5,
                        width=500,
                        height=500,
                        title=("Gregorio Cortez on the run (June 14 to 22, 1901)"),
                        mapbox_style='carto-positron'                               
                        )

fig.show()

In [ ]:
fig.write_html("../corridos/gregmap.html")